# Testarea strategiilor de blackjack cu metoda Monte Carlo

Obiectivul este să simulăm jocul de blackjack (21) în Python, și să folosim metoda Monte Carlo ca să vedem care sunt cele mai bune strategii de joc.

In [1]:
from enum import Enum, auto
import math
import random

random.seed(7)

In [2]:
class Card(Enum):
    "Represents a standard playing card."

    _2 = auto()
    _3 = auto()
    _4 = auto()
    _5 = auto()
    _6 = auto()
    _7 = auto()
    _8 = auto()
    _9 = auto()
    _10 = auto()
    A = auto()
    J = auto()
    Q = auto()
    K = auto()
    
    @property
    def points(self):
        if self == Card._2:
            return 2
        elif self == Card._3:
            return 3
        elif self == Card._4:
            return 4
        elif self == Card._5:
            return 5
        elif self == Card._6:
            return 6
        elif self == Card._7:
            return 7
        elif self == Card._8:
            return 8
        elif self == Card._9:
            return 9
        elif self == Card.A:
            return 11
        else:
            return 10
    
    def __str__(self):
        return self.name.replace('_', '')

    def __repr__(self):
        return f'Card({self})'

In [3]:
class Deck:
    "A standard deck of 52 cards."

    def __init__(self):
        self.cards = []
        
        self.cards += 4 * [Card.A]
        self.cards += 4 * [Card._2]
        self.cards += 4 * [Card._3]
        self.cards += 4 * [Card._4]
        self.cards += 4 * [Card._5]
        self.cards += 4 * [Card._6]
        self.cards += 4 * [Card._7]
        self.cards += 4 * [Card._8]
        self.cards += 4 * [Card._9]
        self.cards += 4 * [Card._10]
        self.cards += 4 * [Card.J]
        self.cards += 4 * [Card.Q]
        self.cards += 4 * [Card.K]
        
        assert self.is_complete(), "Deck is missing cards"
    
    @property
    def num_cards(self):
        return len(self.cards)

    def is_complete(self):
        return self.num_cards == 52
    
    def take_card(self):
        assert self.num_cards > 0, "Deck is empty"
        index = random.randint(0, self.num_cards - 1)
        return self.cards.pop(index)
    
    def take_cards(self, num):
        return [self.take_card() for _ in range(num)]

In [4]:
class Action(Enum):
    "An action the player can take."

    HIT = auto()
    STAND = auto()

In [5]:
class Winner(Enum):
    "The winner of a match"

    PLAYER = auto()
    DEALER = auto()

In [6]:
def hand_value(hand):
    "Computes the value of a given hand."

    total = 0
    num_aces = 0
    for card in hand:
        if card == Card.A:
            num_aces += 1
        
        total += card.points
    
    # Change available aces' value from 11 to 1,
    # until the total is below 21.
    while total > 21 and num_aces > 0:
        total -= 10
        num_aces -= 1
            
    return total

In [7]:
def play_game(strategy):
    "Plays a game of blackjack with the given strategy."

    deck = Deck()
    dealer_hand = deck.take_cards(2)
    dealer_card = dealer_hand[0]
    
    player_hand = deck.take_cards(2)
    
    while True:
        action = strategy(player_hand, dealer_card)
        
        if action == Action.STAND:
            break
    
        new_card = deck.take_card()
        player_hand.append(new_card)
        
        value = hand_value(player_hand)
        if value > 21:
            return Winner.DEALER
        elif value == 21:
            return Winner.PLAYER
    
    if hand_value(player_hand) > hand_value(dealer_hand):
        return Winner.PLAYER
    else:
        return Winner.DEALER

In [8]:
def evaluate(strategy, num_experiments=50_000):
    """Evaluates the performance of a given blackjack strategy
    using the Monte Carlo method.
    """

    player_wins = 0

    for _ in range(num_experiments):
        winner = play_game(strategy)
        if winner == Winner.PLAYER:
            player_wins += 1

    return player_wins / num_experiments

In [9]:
def always_stand(hand, dealer_card):
    "Policy which just stands as soon as the hand is dealt."
    return Action.STAND

evaluate(always_stand)

0.4682

In [10]:
def hit_then_stand(hand, dealer_card):
    "Policy which hits, then immediately stands."
    if len(hand) == 1:
        return Action.HIT
    else:
        return Action.STAND

    
evaluate(hit_then_stand)

0.46758

In [11]:
def hit_if_less_than_dealer(hand, dealer_card):
    """Policy which hits only if the current hand is less than
    the dealer's visible card.
    """

    if hand_value(hand) < dealer_card.points:
        return Action.HIT
    else:
        return Action.STAND

evaluate(hit_if_less_than_dealer)

0.4865